# Diagnostic Plots

This notebook have several diagnostic plots for the Copacabana output of the Buzzard v1.9.8 dataset.

created: May, 2022 <br>
author: Johnny H. Esteves

In [1]:
import numpy as np
import scipy.stats as st

from astropy.table import Table, vstack
from astropy.io.fits import getdata
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns; sns.set(color_codes=True)
# plt.rcParams.update({'font.size': 16})
sns.set_context("paper", font_scale=1.3)
sns.set_style("whitegrid")


## Load Dataset

In [3]:
import sys
sys.path.append("/home/s1/jesteves/git/ccopa/python/")
from main import copacabana

root = '/home/s1/jesteves/git/buzzardAnalysis/mainAnalysis/'
cfg  = root+'config_buzzard_v2.yaml'

copa = copacabana(cfg,dataset='buzzard_v2')

/cvmfs/des.opensciencegrid.org/fnal/anaconda2/envs/des18a/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


master file: 
 /data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00017.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00018.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00019.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00020.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00022.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00032.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00033.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00034.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00035.hdf5
/data/des61.a/data/johnny/Buzzard/Buzzard_v2.0.0/y3/output/tiles/buzzard_v2.0.0_copa_00038.hdf5
/data/des61.a/data/johnny

In [84]:
run_name = u'gauss003_rhod_02Lstar_nfw'
cat0 = copa.load_copa_out('cluster',run_name)
gal = copa.load_copa_out('members',run_name,is_bma=False)

In [85]:
non_empty_halos = (cat0['Ngals_true']>0.)&(cat0['Ngals']>0.)
cat = cat0[non_empty_halos].copy()


In [86]:
# cat = cat[cat['Ngals_true']>=2]


In [87]:
def chunks(ids1, ids2):
    """Yield successive n-sized chunks from data"""
    for id in ids2:
        w, = np.where(ids1 == id)
        yield w

def getTruthTable(gal,cat):
    gal2 = gal[gal['True']==True].copy()

    gal2['Pmem'] = 1.

    #indices = list(chunks(gal2['CID'],cat['CID']))
    #ngals = np.array([np.sum(gal2['Pmem'][idx]) for idx in indices])

    cat2 = cat.copy()
    cat2['Ngals'] = cat['Ngals_true'][:]
    cat2['R200'] = cat['R200_true'][:]
    cat2['Nbkg'] = cat['Nbkg_true'][:]

    return gal2,cat2

In [88]:
gal2, cat2 = getTruthTable(gal,cat)

In [89]:
# match indices
idx = [np.where(cat2["CID"] == cidx)[0] for cidx in cat["CID"] if len(np.where(cat2["CID"] == cidx)[0])>0]
idx = np.stack(idx).ravel()
cat2 = cat2[idx]


In [90]:
def zoffset(z,zcls):
    return (z-zcls)/(1+zcls)

def getIndices(gindices,gkeys,ckeys):
    indicies = np.empty((0),dtype=int)
    indicies_into_cluster = np.empty((0),dtype=int)

    for i in range(ckeys.size):
        idx, = np.where(gkeys==ckeys[i])
        if idx.size>0:
            w2 = np.arange(gindices[idx],gindices[idx+1], 1, dtype=int)
            w = np.full(w2.size,i,dtype=int)

            indicies = np.append(indicies,w2)
            indicies_into_cluster = np.append(indicies_into_cluster,w)

    return indicies,indicies_into_cluster

def set_new_variables(gal, gal2, cat, cat2):    
    ## get the same clustes
    cat = cat.group_by('CID')
    cidx, cidx2 = getIndices(cat.groups.indices,cat.groups.keys['CID'],cat2['CID'])
    cat2 = cat2[cidx2]
    cat = cat[cidx]

    gal = gal.group_by('CID')
    gindices, gkeys = gal.groups.indices, gal.groups.keys['CID']
    gidx, cidx = getIndices(gindices, gkeys, cat['CID'])
    gal = gal[gidx]

    gal['Ngals'] = cat['Ngals_true'][cidx]
    gal['M200'] = cat['M200_true'][cidx]
    gal['Rnorm'] = gal['R']/cat['R200_true'][cidx]

    gal2 = gal2.group_by('CID')
    gidx, cidx = getIndices(gal2.groups.indices,gal2.groups.keys['CID'],cat2['CID'])
    gal2 = gal2[gidx]

    gal2['Ngals'] = cat2['Ngals'][cidx]
    gal2['M200'] = cat2['M200_true'][cidx]
    gal2['Rnorm'] = gal2['R']/cat2['R200_true'][cidx]

    gal['z_offset'] = zoffset(gal['z'],gal['redshift'])
    gal2['z_offset'] = zoffset(gal2['z'],gal2['redshift'])

    # lcolor: 0,1,2,3,4
    color_list = ['g-r','g-i','r-i','r-z','i-z']
    color_index = [[0,1],[0,2],[1,2],[1,3],[2,3]]

    for i,pair_idx in enumerate(color_index):
        i0, i1 = pair_idx
        gal[color_list[i]] = gal['mag'][:,i0]-gal['mag'][:,i1]
        gal2[color_list[i]] = gal2['mag'][:,i0]-gal2['mag'][:,i1]

    #indices = list(chunks(gal['CID'],cat['CID']))
    #indices2 = list(chunks(gal2['CID'],cat['CID']))

    # gal['delta_rs'] = get_delta_color(gal,cat,indices)
    # gal2['delta_rs'] = get_delta_color(gal2,cat,indices2)

    return gal, gal2, cat, cat2

In [91]:
print('defining some variables')
gal, gal2, cat, cat2 = set_new_variables(gal, gal2, cat, cat2)


defining some variables


### Setting Bins

In [92]:
def splitBins(var):
    nmin = np.nanmin(var)
    n25 = np.percentile(var,35)
    n50 = np.nanmedian(var)
    n75 = np.percentile(var,75)
    nmax = np.max(var)
    
    return np.array([nmin,n25,n50,n75,nmax])


In [93]:
print('defining bins')
massBins = splitBins(cat2['M200_true'])
zBins = splitBins(cat['redshift'])
nbins = splitBins(cat2['Ngals'])
nbins[0] = 1

radialBin = np.linspace(0.01,1.01,8)
colorBin = np.linspace(-1.5,0.5,12)
zOffsetBin = np.linspace(-0.2,0.2,20)


defining bins


In [94]:
nbins

array([  1.,  14.,  19.,  37., 246.])

## Plots

In [95]:
from plotsLibrary import generalPlots, clusterPlots, checkPlots, sky_plot

In [96]:
print('Sky Plot')
sky_plot(cat['RA'], cat['DEC'],title='Buzzard Simulation v1.9.8')

Sky Plot


In [97]:
print('Plotting General plots')
allPlots = generalPlots()

print('Scaling Relations')
allPlots.plot_scaling_relation(cat,cat2,kind='richness')

Plotting General plots
Scaling Relations


In [98]:
allPlots.plotResidual(cat,cat2,kind=['richness','z'],bins=zBins)
allPlots.plotResidual(cat,cat2,kind=['richness','mass'],bins=massBins)
allPlots.plotResidual(cat,cat2,kind=['richness','N'],bins=nbins)

here


In [99]:
print('Probability Histograms')
allPlots.plot_grid_histograms(gal)
allPlots.plot_grid_fractions_pmem(gal)


Probability Histograms


In [100]:
opt_tr = allPlots.plot_confusion_matrix(gal,'Pmem',title=None)
print('otp th: %.2f'%opt_tr)
allPlots.plot_roc_curve(gal,'Pmem',opt_tr)
allPlots.plot_precision_recall_vs_threshold(gal,'Pmem',lcol='P_{mem}',title=None)


Normalized confusion matrix
otp th: 0.12


In [101]:
allPlots.plot_purity_completeness(gal,gal2)

allPlots.plot_purity_completeness_threshold(gal,gal2,'Pmem')
allPlots.plot_purity_completeness_threshold(gal,gal2,'Pz')

allPlots.plot_purity_completeness_variable(gal,gal2,radialBin,'R')
# allPlots.plot_purity_completeness_variable(gal,gal2,colorBin,'delta_rs')
allPlots.plot_purity_completeness_variable(gal,gal2,zOffsetBin,'z_offset')

allPlots.plot_purity_completeness_variable(gal,gal2,zBins,'redshift')
allPlots.plot_purity_completeness_variable(gal,gal2,massBins,'M200')
allPlots.plot_purity_completeness_variable(gal,gal2,nbins,'Ngals')


In [21]:
allPlots.plot_validation_pdf_radial(gal,gal2,cat,cat2)
allPlots.plot_validation_pdf_redshift(gal,gal2,cat,cat2)
allPlots.plot_validation_pdf_color(gal,gal2,cat,cat2)

# print('Probabilities')
# allPlots.plot_probabilities_radialPDF(gal,gal2,cat,cat2)
allPlots.plot_probabilities_redshiftPDF(gal,gal2,cat,cat2)


plotsLibrary.py:915: RuntimeWarning: divide by zero encountered in divide
  per_error = np.where(pdf/pdf2>1e6,0.,(pdf-pdf2)/pdf2)
plotsLibrary.py:915: RuntimeWarning: invalid value encountered in divide
  per_error = np.where(pdf/pdf2>1e6,0.,(pdf-pdf2)/pdf2)
plotsLibrary.py:915: RuntimeWarning: invalid value encountered in greater
  per_error = np.where(pdf/pdf2>1e6,0.,(pdf-pdf2)/pdf2)
plotsLibrary.py:922: RuntimeWarning: divide by zero encountered in divide
  sper_error = np.where(sg/sg2>1e6,0.,(sg-sg2)/sg2)
plotsLibrary.py:922: RuntimeWarning: invalid value encountered in divide
  sper_error = np.where(sg/sg2>1e6,0.,(sg-sg2)/sg2)
plotsLibrary.py:922: RuntimeWarning: invalid value encountered in greater
  sper_error = np.where(sg/sg2>1e6,0.,(sg-sg2)/sg2)
/home/s1/jesteves/git/ccopa/python/copac/gaussianKDE.py:22: RuntimeWarning: invalid value encountered in divide
  self.weights = weights / np.sum(weights)
/cvmfs/des.opensciencegrid.org/fnal/anaconda2/envs/des18a/lib/python2.7/site-pa

KeyError: 'delta_rs'